## 00. 라이브러리 

In [1]:
# 자바 버젼 확인 (필수)
!java -version

java version "1.8.0_321"
Java(TM) SE Runtime Environment (build 1.8.0_321-b07)
Java HotSpot(TM) 64-Bit Server VM (build 25.321-b07, mixed mode)


In [ ]:
import pandas as pd
import numpy as np
import regex    #정규표현식 모듈
import re    #정규식 모듈
import gensim    #주제 모델링에 중점을 둔 자연어 처리 모듈
from PyKomoran import *     #형태소 분석 모듈
komoran = Komoran("STABLE")
from tqdm import tqdm     #진행률 확인 모듈
tqdm.pandas() 
import pickle    #data type을 byte 형태로 저장 모듈

import seaborn
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family='Malgun Gothic')

## 01 학습 데이터  #

### 데이터 전처리 ###
천만상상오아시스 (2012~2017)

In [4]:
# 주소 확인
os.chdir("c:/머신러닝/미니프로젝트/데이터")
print(os.getcwd())

c:\머신러닝\미니프로젝트\데이터


In [5]:
train = pd.read_csv('train.csv',sep='\t')
train.head(2)

,글번호,제목,작성일,분류,내용,부서의견
0,150587,횡단보도 사각지대 반사경,2012-10-30,교통,운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대가 형성되어 사람이 보이지 ...,"좋은 제안 주신 whatsmatter 님께 감사드리며, 제안을 검토한 결과를 다음과..."
1,150594,자전거 횡단보도 내려서 끌고가기,2012-10-30,교통,자전거 횡단보도와 자전거 표지를 보면 마치 자전거를 타고 건너도 되는 것처럼 보이고...,- 말씀하신 제안에는 충분히 공감합니다. 다만 교통안전표지(노면표시)는 도로교통법상...


In [6]:
# 컬럼 확인
train.columns

Index(['글번호', '제목', '작성일', '분류', '내용', '부서의견'], dtype='object')

In [7]:
# 민원 분류별 개수
train['분류'].value_counts()

환경    2525
교통    2160
안전    1207
문화    1110
건강     930
주택     817
복지     681
경제     595
여성     504
세금     147
Name: 분류, dtype: int64

In [8]:
# 연도별 민원 개수
train['작성일'] = pd.to_datetime(train['작성일'])
train['작성일'].dt.year.value_counts()

2013.0    3969
2014.0    2268
2015.0    1492
2016.0    1396
2017.0     983
2012.0     568
Name: 작성일, dtype: int64

### 데이터 핸들링

In [9]:
# 컬럼별 결측치
train.isnull().sum()

글번호        0
제목         0
작성일        1
분류         1
내용        21
부서의견    6653
dtype: int64

In [10]:
# '내용' 없는 행 삭제
train = train.dropna(subset=['내용'])

In [11]:
# '내용' 중복 행 삭제
train = train.drop_duplicates(subset='내용')

In [12]:
# 전처리 후 민원 분류 개수 및 연도별 민원 개수 재확인
print('<민원 분류 개수> \n',train['분류'].value_counts())
print('\n<연도별 민원 개수> \n',train['작성일'].dt.year.value_counts())

<민원 분류 개수> 
 환경    2391
교통    1980
안전    1097
문화    1060
건강     864
주택     708
복지     629
경제     541
여성     471
세금     131
Name: 분류, dtype: int64

<연도별 민원 개수> 
 2013    3540
2014    1991
2015    1432
2016    1372
2017     979
2012     558
Name: 작성일, dtype: int64


In [13]:
# 제목 + 내용 통합
train['contents'] = train.apply(lambda x:x['제목']+'\n'+x['내용'], axis = 1)
train['contents']

0        횡단보도 사각지대 반사경\n운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대...
1        자전거 횡단보도 내려서 끌고가기\n자전거 횡단보도와 자전거 표지를 보면 마치 자전거...
2        범죄예방을 위한 투도어현관보조문 설치\n- 주민을 위하여 노력하시는 직원여러분께 감...
3        각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.\n어제 ...
4        예식장내 부대시설인 음식점의 횡포를...\n자녀의 결혼등으로 인한 예식이나 회갑 칠...
                               ...                        
10672    지하철 임산부 지정석 관련\n안녕하세요  수고하십니다.다름이 아니오라 지하철 임산부...
10673    채동선 작곡가의 서울 집을 보존해 주세요.\n서울 성북동의 옛날짜장면집 뒷 골목에 ...
10674    한강및 지천에서의 안전\n시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립...
10675    세빛섬 이전 가능한가 ?\n안녕하세요 ?최근에 노들섬이 2018년 말 경 개발된다는...
10676    한강 시민공원 주차장에 태양광 패널로 지붕을 설치하면 좋겠습니다.\n한강시민공원에서...
Name: contents, Length: 9872, dtype: object

## 형태소 분석
### 사용자 사전 추가

In [14]:
#사용자 사전 등록 
user_dic_path = "C:/머신러닝/미니프로젝트/데이터/dic.txt"

from PyKomoran import *

komoran = Komoran('STABLE')
komoran.set_user_dic(user_dic_path)
komoran.get_plain_text('공공자전거인 따릉이를 타고 출근을 하였다.')

'공공자전거/NNG 이/VCP ㄴ/ETM 따릉이/NNG 를/JKO 타/VV 고/EC 출근/NNG 을/JKO 하/VV 었/EP 다/EF ./SF'

In [15]:
komoran.get_list('강변북로 교통혼잡')

[강변북로/NNG, 교통/NNG, 혼잡/NNG]

In [16]:
komoran.get_nouns('공공자전거인 따릉이를 타고 출근을 하였다.')

['공공자전거', '따릉이', '출근']

In [17]:
# 특수문자 제거 #
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\’\“\”\·\n\r\t■◇◆▶;\xa0]', '', readData).strip()
    return text

#명사, 형용사, 관형사, 동사, 부사, 관형사#
def morp(strings):
   return [w.get_first()+'다' if w.get_second() in ['VV','VA'] else w.get_first() for w in komoran.get_list(cleanText(strings)) if w.get_second() in ['NNP','NNG','MAG','VA','VV','MM']]

In [18]:
morp('혼잡스러운 강변북로를 달리다')

['혼잡', '강변북로', '달리다']

In [19]:
morp('공공자전거인 따릉이를 타고 출근을 하였다.')

['공공자전거', '따릉이', '타다', '출근', '하다']

In [34]:
morp('시립도서관과 구립도서관의 차이는?')

['시립도서관', '구립도서관', '차이']

### 토크나이징

In [21]:
# AttributeError: 'Series' object has no attribute 'progress_map' 오류 해결
tqdm.pandas()

In [22]:
# 특정 품사 토크나이징
train['tokens'] = train['contents'].progress_map(lambda x:morp(x))

100%|██████████| 9872/9872 [15:08<00:00, 10.87it/s]


In [23]:
# 모든 품사 토크나이징
train['all_tokens'] = train['contents'].progress_map(lambda x:komoran.get_morphes_by_tags(cleanText(x)))

100%|██████████| 9872/9872 [04:57<00:00, 33.16it/s]


### 불용어(stopwords) 제거 ###

In [35]:
# 불용어 리스트 만들기
stopwords = ['아래', '상상', '제안', '까지', '닷컴', '포털', '사이트', '천만', '오아시스', '이벤트', '접수','서울시','서울','특별시',
             '천만상상','파일','첨부','응모','슬로건','공모','공모전','응모전','신청','경우','때문','정도','사항',
                   '해당','겁니다','이것','저것','그것','돋움','신명', '태명', '한컴', '돋움',
                   '동안','거기','저기','여기','대부분','누구','무엇','고딕','만큼','굴림','감사','건지','텐데',
                   '안녕','안녕하세요','이번','걸로','수고','겁니까','그간','그건','그때','글쓴이','누가','니다','다면',
                   '뭔가','상상오아시스','하다','이다','되다','같다','궁','자체','서체','정','서','이','을','있다','없다', '체','관련',
                   '생각', '현재', '진행', '사람', '마음', '남산', '내용', '현실','음','막','김','변','조',
                   '오','참','동','지금','주변','대상','부분','요즘','하루','마련','시간','이상','행위',
                   '활동','구분','사실','과정','모습','기간','선정','단지','자신','발생','지역','기대','마련',
                   '장소','모두','부탁','제공','이용','해주','당시','최근','민원','문제','문제점','현황','개선','방안',
                   '문의','답변','일동','요청','담당자','직원','방법','사용','활용','확인','방식','예전','안녕하십니까',
                   '이하','바로 가기','바로가기','제가','먹다','보이다','지나다','콜론','각','대하다','따르다','인하다','전','후','예','한','다','보다','그렇다','좋다',
                   '나다','너무','정말','내다','많다','그리다','물론','조금','다시','느끼다','없이','특히','여러','다른','이런','가지다','어떻다','통하다', '위하다']
                   
# 기존 불용어 리스트에 추가
stopwords.append('홈페이지')
stopwords_set = set(stopwords)

In [36]:
# 불용어 제거
train['tokens'] = train['tokens'].map(lambda x:[w for w in x if not w in stopwords_set])
train['all_tokens'] = train['all_tokens'].map(lambda x:[w for w in x if not w in stopwords_set])

In [37]:
train['tokens']

0       [횡단보도, 사각지대, 반사경, 운전, 횡단보도, 적치, 물, 사각지대, 형성, 때...
1       [자전거, 횡단보도, 내리다, 끌다, 자전거, 횡단보도, 자전거, 표지, 마치, 자...
2       [범죄, 예방, 위하다, 도어, 현관, 보조, 문, 설치, 주민, 위하다, 노력, ...
3                                                      []
4       [예식장, 부대시설, 음식점, 횡포, 자녀, 결혼, 예식, 회갑, 웨딩, 홀, 호텔...
                              ...                        
9837    [버스, 노약자, 보호, 석, 내리다, 문, 곳, 내리다, 문, 가깝다, 곳, 지정...
9838    [지하철, 임산부, 지정석, 다르다, 오라, 지하철, 임산부, 지정석, 건의, 드리...
9839    [동서, 작곡가, 집, 보존, 성북동, 옛날, 짜장면, 집, 뒤, 골목, 한옥, 집...
9840    [한강, 및, 지천, 안전, 시민, 안전, 편의, 위하다, 불철주야, 감사드리다, ...
9841    [한강, 시민공원, 주차장, 태양광, 패널, 지붕, 설치, 한강시민공원, 주차장, ...
Name: tokens, Length: 9842, dtype: object

In [62]:
## 단어 확인용 코드
n = 0
for i in range(0,len(train)):
    if "자전" in train['tokens'][i]:
        print(train['tokens'][i])
        n += 1
print(n)

['자전거', '도로', '일반', '인도', '자전거', '타다', '진입', '금지', '현행', '법규', '자전거', '차량', '교통수단', '차량', '자전거', '타다', '다니다', '교통법', '규', '적용', '받다', '자전거', '타다', '교통법', '규', '적용', '받다', '타다', '다니다', '자전거', '도로', '타다', '다니다', '간혹', '자전거', '도로', '곳', '자전거', '타다', '다니다', '계시다', '자전거', '도로', '인도', '오다', '타다', '다니다', '시민', '편하다', '걸다', '자전거', '오다', '비키다', '달다', '기분', '매우', '바쁘다', '빨리', '가다', '좀', '비키다', '달다', '엄연히', '인도', '인도', '만약', '사고', '잘', '못', '자전거', '타다', '책임', '도로교통', '법', '위반', '뉴스', '간혹', '나오다', '속도', '줄이다', '안전모', '착용', '자전', '도로', '다니다', '지키다', '처음', '그렇게', '모르다', '지키다', '많이', '자꾸', '이렇다', '조례', '인도', '자전거', '타다', '금지', '만들다', '타다', '이런저런', '말', '자꾸', '나오다', '관계', '말', '아예', '자전거', '인도', '진입', '금지', '법', '대안']
['주요', '사무', '업무', '빌딩', '혹은', '내부', '자전거', '전용', '주차장', '설치', '주요', '사무', '업무', '빌딩', '근방', '자전거', '주차', '공간', '부족', '실정', '자전거', '인구', '시대', '자전거', '이용자', '고가', '자전거', '자', '출', '족', '자전거', '출퇴근', '족', '늘어나다', '다만', '시내', '안심', '자전거', '주차', '공간', '중구', '교통', '관리과', '기업체', '교통', '수요', '

In [39]:
train

,글번호,제목,작성일,분류,내용,부서의견,contents,tokens,all_tokens
0,150587,횡단보도 사각지대 반사경,2012-10-30,교통,운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대가 형성되어 사람이 보이지 ...,"좋은 제안 주신 whatsmatter 님께 감사드리며, 제안을 검토한 결과를 다음과...",횡단보도 사각지대 반사경\n운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대...,"[횡단보도, 사각지대, 반사경, 운전, 횡단보도, 적치, 물, 사각지대, 형성, 때...","[횡단보도, 사각지대, 반사경, 운전, 하, 보, 면, 횡단보도, 에, 적치, 물,..."
1,150594,자전거 횡단보도 내려서 끌고가기,2012-10-30,교통,자전거 횡단보도와 자전거 표지를 보면 마치 자전거를 타고 건너도 되는 것처럼 보이고...,- 말씀하신 제안에는 충분히 공감합니다. 다만 교통안전표지(노면표시)는 도로교통법상...,자전거 횡단보도 내려서 끌고가기\n자전거 횡단보도와 자전거 표지를 보면 마치 자전거...,"[자전거, 횡단보도, 내리다, 끌다, 자전거, 횡단보도, 자전거, 표지, 마치, 자...","[자전거, 횡단보도, 내리, 어서, 끌, 고, 가, 기, 자전거, 횡단보도, 와, ..."
2,150597,범죄예방을 위한 투도어현관보조문 설치,2012-10-30,주택,- 주민을 위하여 노력하시는 직원여러분께 감사드립니다 - 아파트및 다세대 등 홀로 ...,"000님 안녕하세요? 000님께서 공동주택(아파트, 다세대, 연립)의 현관문에 1인...",범죄예방을 위한 투도어현관보조문 설치\n- 주민을 위하여 노력하시는 직원여러분께 감...,"[범죄, 예방, 위하다, 도어, 현관, 보조, 문, 설치, 주민, 위하다, 노력, ...","[범죄, 예방, 위하, ㄴ, 투, 도어, 현관, 보조, 문, 설치, 주민, 위하, ..."
3,150659,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.,2012-10-31,안전,어제 어린 남매 두명이 집에 전자렌지로 인해 불이나서중태에 빠졌다는 기사를 보았습니...,기시행중임 -소방시설설치유지 및 안전관리에 관한 법률 제8조제1항 및 서울특별시 주...,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.\n어제 ...,[],[각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다어제 어린...
4,150660,예식장내 부대시설인 음식점의 횡포를...,2012-10-31,경제,자녀의 결혼등으로 인한 예식이나 회갑 칠순등으로 인하여 웨딩홀이나 호텔등을 이용시에...,안녕하십니까? 우리시정에 큰 관심을 보내주신 데 대하여 깊은 감사드립니다. dski...,예식장내 부대시설인 음식점의 횡포를...\n자녀의 결혼등으로 인한 예식이나 회갑 칠...,"[예식장, 부대시설, 음식점, 횡포, 자녀, 결혼, 예식, 회갑, 웨딩, 홀, 호텔...","[예식장, 내, 부대시설, ㄴ, 음식점, 의, 횡포, 를, 자녀, 의, 결혼, 등,..."
...,...,...,...,...,...,...,...,...,...
9837,179105,"버스 노약자보호석을 내리는 문이 보이는곳, 내리는문에 가까운 곳에 지정",2017-08-31,교통,보통 버스안에 보면 노약자 보호석은 타는 쪽 좌석에 지정되어 있고 내리는 문에서는 ...,NaN,"버스 노약자보호석을 내리는 문이 보이는곳, 내리는문에 가까운 곳에 지정\n보통 버스...","[버스, 노약자, 보호, 석, 내리다, 문, 곳, 내리다, 문, 가깝다, 곳, 지정...","[버스, 노약자, 보호, 석, 내리, 는, 문, 보이, 는, 곳, 내리, 는, 문,..."
9838,179106,지하철 임산부 지정석 관련,2017-08-31,교통,안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부지정석 관련 건의 드립니다.현...,김혁님 안녕하십니까? 제안 참여에 감사드리며 “지하철 임산부 지정석”건에 대하여 답...,지하철 임산부 지정석 관련\n안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부...,"[지하철, 임산부, 지정석, 다르다, 오라, 지하철, 임산부, 지정석, 건의, 드리...","[지하철, 임산부, 지정석, 하, 시, ㅂ니다, 다르, ㅁ, 아니, 오라, 지하철,..."
9839,179107,채동선 작곡가의 서울 집을 보존해 주세요.,2017-08-31,문화,서울 성북동의 옛날짜장면집 뒷 골목에 한옥집들 사이로 고풍스러운 2층 적산가옥이 보...,NaN,채동선 작곡가의 서울 집을 보존해 주세요.\n서울 성북동의 옛날짜장면집 뒷 골목에 ...,"[동서, 작곡가, 집, 보존, 성북동, 옛날, 짜장면, 집, 뒤, 골목, 한옥, 집...","[채, 동서, ㄴ, 작곡가, 의, 집, 보존, 하, 아, 주, 시, 어요, 성북동,..."
9840,179108,한강및 지천에서의 안전,2017-08-31,환경,시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립니다.저는 주로 한강및 한...,NaN,한강및 지천에서의 안전\n시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립...,"[한강, 및, 지천, 안전, 시민, 안전, 편의, 위하다, 불철주야, 감사드리다, ...","[한강, 및, 지천, 에서, 의, 안전, 시민, 의, 안전, 과, 편의, 를, 위하..."


In [40]:
# 토크나이징 후에도 중복된 행 제거
train["res"] = train["tokens"].astype(str) #list to string
train = train.drop_duplicates(subset="res")
train = train.drop('res',1)
train = train.reset_index(drop=True) # index 리셋

C:\Users\dkq50\AppData\Local\Temp/ipykernel_9360/2713820896.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train = train.drop('res',1)


In [41]:
train

,글번호,제목,작성일,분류,내용,부서의견,contents,tokens,all_tokens
0,150587,횡단보도 사각지대 반사경,2012-10-30,교통,운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대가 형성되어 사람이 보이지 ...,"좋은 제안 주신 whatsmatter 님께 감사드리며, 제안을 검토한 결과를 다음과...",횡단보도 사각지대 반사경\n운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대...,"[횡단보도, 사각지대, 반사경, 운전, 횡단보도, 적치, 물, 사각지대, 형성, 때...","[횡단보도, 사각지대, 반사경, 운전, 하, 보, 면, 횡단보도, 에, 적치, 물,..."
1,150594,자전거 횡단보도 내려서 끌고가기,2012-10-30,교통,자전거 횡단보도와 자전거 표지를 보면 마치 자전거를 타고 건너도 되는 것처럼 보이고...,- 말씀하신 제안에는 충분히 공감합니다. 다만 교통안전표지(노면표시)는 도로교통법상...,자전거 횡단보도 내려서 끌고가기\n자전거 횡단보도와 자전거 표지를 보면 마치 자전거...,"[자전거, 횡단보도, 내리다, 끌다, 자전거, 횡단보도, 자전거, 표지, 마치, 자...","[자전거, 횡단보도, 내리, 어서, 끌, 고, 가, 기, 자전거, 횡단보도, 와, ..."
2,150597,범죄예방을 위한 투도어현관보조문 설치,2012-10-30,주택,- 주민을 위하여 노력하시는 직원여러분께 감사드립니다 - 아파트및 다세대 등 홀로 ...,"000님 안녕하세요? 000님께서 공동주택(아파트, 다세대, 연립)의 현관문에 1인...",범죄예방을 위한 투도어현관보조문 설치\n- 주민을 위하여 노력하시는 직원여러분께 감...,"[범죄, 예방, 위하다, 도어, 현관, 보조, 문, 설치, 주민, 위하다, 노력, ...","[범죄, 예방, 위하, ㄴ, 투, 도어, 현관, 보조, 문, 설치, 주민, 위하, ..."
3,150659,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.,2012-10-31,안전,어제 어린 남매 두명이 집에 전자렌지로 인해 불이나서중태에 빠졌다는 기사를 보았습니...,기시행중임 -소방시설설치유지 및 안전관리에 관한 법률 제8조제1항 및 서울특별시 주...,각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다.\n어제 ...,[],[각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다어제 어린...
4,150660,예식장내 부대시설인 음식점의 횡포를...,2012-10-31,경제,자녀의 결혼등으로 인한 예식이나 회갑 칠순등으로 인하여 웨딩홀이나 호텔등을 이용시에...,안녕하십니까? 우리시정에 큰 관심을 보내주신 데 대하여 깊은 감사드립니다. dski...,예식장내 부대시설인 음식점의 횡포를...\n자녀의 결혼등으로 인한 예식이나 회갑 칠...,"[예식장, 부대시설, 음식점, 횡포, 자녀, 결혼, 예식, 회갑, 웨딩, 홀, 호텔...","[예식장, 내, 부대시설, ㄴ, 음식점, 의, 횡포, 를, 자녀, 의, 결혼, 등,..."
...,...,...,...,...,...,...,...,...,...
9837,179105,"버스 노약자보호석을 내리는 문이 보이는곳, 내리는문에 가까운 곳에 지정",2017-08-31,교통,보통 버스안에 보면 노약자 보호석은 타는 쪽 좌석에 지정되어 있고 내리는 문에서는 ...,NaN,"버스 노약자보호석을 내리는 문이 보이는곳, 내리는문에 가까운 곳에 지정\n보통 버스...","[버스, 노약자, 보호, 석, 내리다, 문, 곳, 내리다, 문, 가깝다, 곳, 지정...","[버스, 노약자, 보호, 석, 내리, 는, 문, 보이, 는, 곳, 내리, 는, 문,..."
9838,179106,지하철 임산부 지정석 관련,2017-08-31,교통,안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부지정석 관련 건의 드립니다.현...,김혁님 안녕하십니까? 제안 참여에 감사드리며 “지하철 임산부 지정석”건에 대하여 답...,지하철 임산부 지정석 관련\n안녕하세요 수고하십니다.다름이 아니오라 지하철 임산부...,"[지하철, 임산부, 지정석, 다르다, 오라, 지하철, 임산부, 지정석, 건의, 드리...","[지하철, 임산부, 지정석, 하, 시, ㅂ니다, 다르, ㅁ, 아니, 오라, 지하철,..."
9839,179107,채동선 작곡가의 서울 집을 보존해 주세요.,2017-08-31,문화,서울 성북동의 옛날짜장면집 뒷 골목에 한옥집들 사이로 고풍스러운 2층 적산가옥이 보...,NaN,채동선 작곡가의 서울 집을 보존해 주세요.\n서울 성북동의 옛날짜장면집 뒷 골목에 ...,"[동서, 작곡가, 집, 보존, 성북동, 옛날, 짜장면, 집, 뒤, 골목, 한옥, 집...","[채, 동서, ㄴ, 작곡가, 의, 집, 보존, 하, 아, 주, 시, 어요, 성북동,..."
9840,179108,한강및 지천에서의 안전,2017-08-31,환경,시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립니다.저는 주로 한강및 한...,NaN,한강및 지천에서의 안전\n시민의 안전과 편의를 위하여 불철주야 수고 하심에 감사드립...,"[한강, 및, 지천, 안전, 시민, 안전, 편의, 위하다, 불철주야, 감사드리다, ...","[한강, 및, 지천, 에서, 의, 안전, 시민, 의, 안전, 과, 편의, 를, 위하..."


##  02 테스트 데이터
### 민주주의 서울 자유제안
http://data.seoul.go.kr/dataList/OA-2563/S/1/datasetView.do;jsessionid=B1523D9E9088AF270D511C83450EFAE3.new_portal-svr-21
* 천만상상오아시스 학습데이터와 달리 민원 분류 컬럼이 없음
* 학습데이터를 활용 해 테스트 데이터에 민원 분류를 자동으로 진행하고자 함

In [107]:
test1 = pd.read_excel("민주주의 서울 자유제안 정보_17_19.xlsx")
test1 # 2017년12월18일 ~ 2019년12월18일까지

C:\Users\dkq50\anaconda3\envs\hamtori\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,제안번호,제안제목,제안내용,득표,제안등록일자
0,186203,중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?,"<span style=""font-size: 12px;"">중금속 문제가 심각해지고 있...",0,2019-12-18 01:21:40.0
1,186202,제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주새요,제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...,0,2019-12-18 00:41:00.0
2,186201,"지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대",서울시의 경우 지하철이 상당히 발달되어 있어 교통서비스를 제공하고 있습니다.&nbs...,0,2019-12-17 13:41:34.0
3,186200,재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요,몇몇구에서는 재활용 쓰레기 분리수거함을 원룸촌에 제공해주는 것으로 알고있는데요. 대...,0,2019-12-17 11:10:03.0
4,186199,임산부석을 마냥 비워두지 않는 방법,<p>저는 지하철을 자주 애용하는 여성입니다.</p><p>임산부석이 지정되어 있어 ...,0,2019-12-17 10:55:54.0
...,...,...,...,...,...
4319,180039,버스 정류장 근처의 빛 공해를 해결해 주세요 !,&nbsp;강동구 5호선 명일역 3번 출구에서 올라오면&nbsp; &nbsp;바로&...,0,2017-12-22 16:37:33.0
4320,180038,서울시내버스 안내방송을 부탁드려요 !,&nbsp;서울 시내 버스를 자주 이용하는 편입니다.&nbsp; 버스 내부에는 주로...,1,2017-12-21 18:44:46.0
4321,180037,서울역 인근 버스정류장 명 병행표기 제안.,<p>안녕하세요.</p><p>버스를 타고 서울역을 자주 지나는데 항상 이에 의문을 ...,1,2017-12-21 11:13:14.0
4322,180036,따릉이 보관소 설치 제안,서초역 주변에 따릉이 보관소 설치를 제안합니다!,0,2017-12-20 18:54:23.0


### 데이터 전처리 (Test1)
#### 데이터핸들링

In [87]:
# 컬럼별 결측치 확인
test1.isnull().sum()

제안번호       0
제안제목       0
제안내용      96
득표         0
제안등록일자     0
dtype: int64

In [88]:
# '제안내용' 빈칸인 행 삭제
test1 = test1.dropna(subset=['제안내용'])

In [89]:
# '제안내용' 중복 행 삭제
test1 = test1.drop_duplicates(subset=['제안내용'])

In [90]:
# '제안제목' + '제안내용' = contents 컬럼
test1['contents'] = test1.apply(lambda x:x['제안제목']+'\n'+x['제안내용'],axis=1)
test1['contents']

0       중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?\n<sp...
1       제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...
2       지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대\n서울시의 경우 지하철이...
3       재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요\n몇몇구에서는 재활용 쓰...
4       임산부석을 마냥 비워두지 않는 방법\n<p>저는 지하철을 자주 애용하는 여성입니다....
                              ...                        
4319    버스 정류장 근처의 빛  공해를   해결해 주세요  !\n&nbsp;강동구 5호선 ...
4320    서울시내버스 안내방송을 부탁드려요 !\n&nbsp;서울 시내 버스를 자주 이용하는 ...
4321    서울역 인근 버스정류장 명 병행표기 제안.\n<p>안녕하세요.</p><p>버스를 타...
4322            따릉이 보관소 설치 제안\n서초역 주변에 따릉이 보관소 설치를 제안합니다!
4323    이륜차(오토바이)업소 주변 행정지도 통해 거리질서 확립필요\n<p>서울시내 어디를 ...
Name: contents, Length: 4195, dtype: object

In [91]:
# '제안등록일자' 컬럼 시계열 데이터 타입으로 변경
test1['제안등록일자'] = pd.to_datetime(test1['제안등록일자'])
test1['제안등록일자']

0      2019-12-18 01:21:40
1      2019-12-18 00:41:00
2      2019-12-17 13:41:34
3      2019-12-17 11:10:03
4      2019-12-17 10:55:54
               ...        
4319   2017-12-22 16:37:33
4320   2017-12-21 18:44:46
4321   2017-12-21 11:13:14
4322   2017-12-20 18:54:23
4323   2017-12-18 20:33:46
Name: 제안등록일자, Length: 4195, dtype: datetime64[ns]

### 텍스트 전처리

In [92]:
# html tag 내 한글 제거
def remove_tag(content):
    cleaner = re.compile('<.*?>')
    cleantext = re.sub(cleaner, '', content)
    return cleantext    

In [93]:
# 태그 패턴
tag_pattern = regex.compile(r'class\=\'*\p{Hangul}+\"*|font-family\: \"*p{Hangul}+\"*|HY중고딕|서울남산체|맑은 고딕|함초롬바탕|굴림|굴림체|새굴림|고딕|나눔고딕|산돌고딕|모던고딕|한양중고딕|HY견명조|바탕|태그래픽|궁서|궁서체|휴먼 명조|휴먼명조|\&nbsp\;')

In [94]:
# 여러 html 태그 패턴 삭제
test1['contents'] = test1['contents'].map(lambda x:regex.sub(tag_pattern,'',remove_tag(x)))

In [95]:
test1['contents']

0       중금속 문제가 심각해지고 있는데 중금속으로 분리할수 있는 방법은 없을까요?\n중금속...
1       제설제에 사용되는 cacl2에서 염소가 환경오염을 일으킨다는데 친환경적인걸로 바꿔주...
2       지하철역 위의 임대아파트 건설 후 청년, 신혼부분에게 임대\n서울시의 경우 지하철이...
3       재활용 정거장이나 원룸에 재활용 쓰레기통 지원을 늘려주세요\n몇몇구에서는 재활용 쓰...
4       임산부석을 마냥 비워두지 않는 방법\n저는 지하철을 자주 애용하는 여성입니다.임산부...
                              ...                        
4319    버스 정류장 근처의 빛  공해를   해결해 주세요  !\n강동구 5호선 명일역 3번...
4320    서울시내버스 안내방송을 부탁드려요 !\n서울 시내 버스를 자주 이용하는 편입니다. ...
4321    서울역 인근 버스정류장 명 병행표기 제안.\n안녕하세요.버스를 타고 서울역을 자주 ...
4322            따릉이 보관소 설치 제안\n서초역 주변에 따릉이 보관소 설치를 제안합니다!
4323    이륜차(오토바이)업소 주변 행정지도 통해 거리질서 확립필요\n서울시내 어디를 가보든...
Name: contents, Length: 4195, dtype: object

### 형태소 분석
#### 토크나이징

In [96]:
# 특정 품사 토크나이징
test1['tokens'] = test1['contents'].progress_map(lambda x:morp(x))

100%|██████████| 4195/4195 [04:07<00:00, 16.92it/s]


In [97]:
# 모든 품사 토크나이징
test1['all_tokens'] = test1['contents'].progress_map(lambda x:komoran.get_morphes_by_tags(cleanText(x)))

100%|██████████| 4195/4195 [01:11<00:00, 58.36it/s] 


### 불용어(Stopwords) 제거 ###

In [100]:
test1['tokens'] = test1['tokens'].map(lambda x:[w for w in x if not w in stopwords_set])
test1['all_tokens'] = test1['all_tokens'].map(lambda x:[w for w in x if not w in stopwords_set])

#### Pickle 파일 만들기

* pickle은 텍스트 상태의 데이터가 아닌 파이썬 객체 자체를 바이너리 파일로 저장하는 것이라고 한다.

* 즉, 텍스트 형태로 파일을 저장하는 것이 아니라 dictionary, list, tuple과 같은 형태로 필요한 부분을 저장하는 것이다. 하지만 문자열이 아닌 객체를 파일에 쓸 수 없기 때문에 pickle 모듈을 활용해 그 객체 자체를 바이너리로 저장하는 것

* 장점
    * 빠르다text 파일을 사용하는 경우 필요한 부분들을 파싱해야 하지만 pickle은 이미 필요한 형태대로 저장이 되어 있기 때문에 훨씬 빠르다고 한다.
 

* 주의점
    * 안전하지 않다 pickle module은 안전하지 않고 unpickle data만 신뢰할 수 있다고 한다.RCE(원격코드실행) 공격을 받을 수 있어, 다운 받은 pickle 파일을 사용할 때 매우 주의해야 한다.
 
    * 검증 되지 않은 pickle data를 unpicking하게 될 때 임의의 코드가 실행될 수 있다고 한다.

In [102]:
sample_list = [train, test]
sample_list

[          글번호                                         제목        작성일  분류  \
 0      150587                              횡단보도 사각지대 반사경 2012-10-30  교통   
 1      150594                          자전거 횡단보도 내려서 끌고가기 2012-10-30  교통   
 2      150597                       범죄예방을 위한 투도어현관보조문 설치 2012-10-30  주택   
 3      150659  각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다. 2012-10-31  안전   
 4      150660                     예식장내 부대시설인 음식점의 횡포를... 2012-10-31  경제   
 ...       ...                                        ...        ...  ..   
 10672  179106                             지하철 임산부 지정석 관련 2017-08-31  교통   
 10673  179107                    채동선 작곡가의 서울 집을 보존해 주세요. 2017-08-31  문화   
 10674  179108                               한강및 지천에서의 안전 2017-08-31  환경   
 10675  179109                              세빛섬 이전 가능한가 ? 2017-08-31  환경   
 10676  179110       한강 시민공원 주차장에 태양광 패널로 지붕을 설치하면 좋겠습니다. 2017-08-31  환경   
 
                                                       내용  \
 0      운전을 하다보면 횡단보도 주변에

In [104]:
## Save pickle
with open("sample.pickle","wb") as fw:
    pickle.dump([train, test1], fw)


### 데이터 전처리 (Test2)
#### 데이터핸들링

In [42]:
test2 = pd.read_csv("민주주의 서울 자유제안 정보_20_22.csv",encoding='cp949')
test2.head(2) # 코로나 이후인 2020년 3월10일 ~ 2022년 3월7일까지

,제안번호,제안제목,제안내용,제안등록일자
0,192222,북부 직업훈련원 본부 장애인 화장실 문고리 중앙으로,http://democracy.seoul.go.kr/front/freeSuggest...,2022-03-10 22:40:32.0
1,192221,수도요금 고지서를 개선하여 시민들에게 납부편의(식별용이)를 제공해 주세요!,http://democracy.seoul.go.kr/front/freeSuggest...,2022-03-10 22:22:22.0


In [45]:
# 컬럼별 결측치 확인
test2.isnull().sum()

제안번호      0
제안제목      0
제안내용      0
제안등록일자    0
dtype: int64

In [47]:
# '제안내용' 중복 행 삭제
test2 = test2.drop_duplicates(subset=['제안제목'])

제안내용이 없으므로 합치는 순서는 생략!

In [49]:
# '제안등록일자' 컬럼 시계열 데이터 타입으로 변경
test2['제안등록일자'] = pd.to_datetime(test2['제안등록일자'])
test2['제안등록일자']

C:\Users\dkq50\AppData\Local\Temp/ipykernel_9360/1540787588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['제안등록일자'] = pd.to_datetime(test2['제안등록일자'])


0      2022-03-10 22:40:32
1      2022-03-10 22:22:22
2      2022-03-10 19:10:58
3      2022-03-10 18:20:48
4      2022-03-10 07:29:25
               ...        
4883   2020-03-11 10:14:32
4884   2020-03-11 10:00:19
4885   2020-03-11 09:58:25
4886   2020-03-11 09:56:02
4887   2020-03-11 09:46:58
Name: 제안등록일자, Length: 4814, dtype: datetime64[ns]

### 텍스트 전처리

In [50]:
# html tag 내 한글 제거
def remove_tag(content):
    cleaner = re.compile('<.*?>')
    cleantext = re.sub(cleaner, '', content)
    return cleantext    

In [51]:
# 태그 패턴
tag_pattern = regex.compile(r'class\=\'*\p{Hangul}+\"*|font-family\: \"*p{Hangul}+\"*|HY중고딕|서울남산체|맑은 고딕|함초롬바탕|굴림|굴림체|새굴림|고딕|나눔고딕|산돌고딕|모던고딕|한양중고딕|HY견명조|바탕|태그래픽|궁서|궁서체|휴먼 명조|휴먼명조|\&nbsp\;')

In [53]:
# 여러 html 태그 패턴 삭제
test2['contents'] = test2['제안제목'].map(lambda x:regex.sub(tag_pattern,'',remove_tag(x)))

In [54]:
test2['contents']

0                  북부 직업훈련원  본부 장애인 화장실  문고리 중앙으로
1       수도요금 고지서를 개선하여 시민들에게 납부편의(식별용이)를 제공해 주세요!
2                 환경미화원 안전과 인권을 위해 작업시간 변경을 요청합니다
3                           비리가 없는 공무직 채용으로 바꿔주세요
4                       희년제도부활 희년자유경제특구개발130. 5개년
                          ...                    
4883                      코로나로 인해 사회멈춤이 해결방안이 맞나요
4884                       코로나 바이러스로 교회예배를 금지해주세요
4885                             마스크 5부제 수량 증가 요청
4886                                콜센타인근학원휴강권고요청
4887                               서울 전국 콜센터 관련하여
Name: contents, Length: 4814, dtype: object

### 형태소 분석
#### 토크나이징

In [55]:
# 특정 품사 토크나이징
test2['tokens'] = test2['contents'].progress_map(lambda x:morp(x))

100%|██████████| 4814/4814 [00:16<00:00, 293.89it/s]


In [57]:
# 모든 품사 토크나이징
test2['all_tokens'] = test2['contents'].progress_map(lambda x:komoran.get_morphes_by_tags(cleanText(x)))

100%|██████████| 4814/4814 [00:22<00:00, 214.36it/s]


### 불용어(Stopwords) 제거 ###

In [58]:
test2['tokens'] = test2['tokens'].map(lambda x:[w for w in x if not w in stopwords_set])
test2['all_tokens'] = test2['all_tokens'].map(lambda x:[w for w in x if not w in stopwords_set])

In [59]:
test2

,제안번호,제안제목,제안내용,제안등록일자,contents,tokens,all_tokens
0,192222,북부 직업훈련원 본부 장애인 화장실 문고리 중앙으로,http://democracy.seoul.go.kr/front/freeSuggest...,2022-03-10 22:40:32,북부 직업훈련원 본부 장애인 화장실 문고리 중앙으로,"[북부, 직업, 훈련원, 본부, 장애인, 화장실, 문고리, 중앙]","[북부, 직업, 훈련원, , 본부, 장애인, 화장실, , 문고리, 중앙, 으로]"
1,192221,수도요금 고지서를 개선하여 시민들에게 납부편의(식별용이)를 제공해 주세요!,http://democracy.seoul.go.kr/front/freeSuggest...,2022-03-10 22:22:22,수도요금 고지서를 개선하여 시민들에게 납부편의(식별용이)를 제공해 주세요!,"[수도, 요금, 고지서, 시민, 납부, 편의, 식별, 용이]","[수도, 요금, 고지서, 를, 하, 아, 시민, 들, 에게, 납부, 편의, 식별, ..."
2,192220,환경미화원 안전과 인권을 위해 작업시간 변경을 요청합니다,http://democracy.seoul.go.kr/front/freeSuggest...,2022-03-10 19:10:58,환경미화원 안전과 인권을 위해 작업시간 변경을 요청합니다,"[환경미화, 안전, 인권, 위하다, 작업, 변경]","[환경미화, 원, 안전, 과, 인권, 위하, 아, 작업, 변경, 하, ㅂ니다]"
3,192219,비리가 없는 공무직 채용으로 바꿔주세요,http://democracy.seoul.go.kr/front/freeSuggest...,2022-03-10 18:20:48,비리가 없는 공무직 채용으로 바꿔주세요,"[비리, 공무, 채용, 바꾸다]","[비리, 가, 없, 는, 공무, 직, 채용, 으로, 바꾸, 어, 주, 시, 어요]"
4,192218,희년제도부활 희년자유경제특구개발130. 5개년,http://democracy.seoul.go.kr/front/freeSuggest...,2022-03-10 07:29:25,희년제도부활 희년자유경제특구개발130. 5개년,"[희년, 부활, 희년, 자유, 경제특구, 개발]","[희년, 제, 도, 부활, 희년, 자유, 경제특구, 개발, 130, 5, 개년]"
...,...,...,...,...,...,...,...
4883,186616,코로나로 인해 사회멈춤이 해결방안이 맞나요,http://democracy.seoul.go.kr/front/freeSuggest...,2020-03-11 10:14:32,코로나로 인해 사회멈춤이 해결방안이 맞나요,"[코로나, 사회, 멈추다, 해결방안, 맞다]","[코로나, 로, 인하, 아, 사회, 멈추, ㅁ, 해결방안, 맞, 나요]"
4884,186615,코로나 바이러스로 교회예배를 금지해주세요,http://democracy.seoul.go.kr/front/freeSuggest...,2020-03-11 10:00:19,코로나 바이러스로 교회예배를 금지해주세요,"[코로나, 바이러스, 교회, 예배, 금지]","[코로나, 바이러스, 로, 교회, 예배, 를, 금지, 하, 아, 주, 시, 어요]"
4885,186614,마스크 5부제 수량 증가 요청,http://democracy.seoul.go.kr/front/freeSuggest...,2020-03-11 09:58:25,마스크 5부제 수량 증가 요청,"[마스크, 수량, 증가]","[마스크, 5, 부제, 수량, 증가]"
4886,186613,콜센타인근학원휴강권고요청,http://democracy.seoul.go.kr/front/freeSuggest...,2020-03-11 09:56:02,콜센타인근학원휴강권고요청,"[콜, 센타, 인근, 학원, 휴강, 권고]","[콜, 센타, 인근, 학원, 휴강, 권고]"


#### Pickle 파일 만들기

In [60]:
sample_list = [train, test2]
sample_list

[         글번호                                         제목        작성일  분류  \
 0     150587                              횡단보도 사각지대 반사경 2012-10-30  교통   
 1     150594                          자전거 횡단보도 내려서 끌고가기 2012-10-30  교통   
 2     150597                       범죄예방을 위한 투도어현관보조문 설치 2012-10-30  주택   
 3     150659  각각 집마다 간이소화기를 만들어 배치하는 것을 의무로 하였으면 좋겠습니다. 2012-10-31  안전   
 4     150660                     예식장내 부대시설인 음식점의 횡포를... 2012-10-31  경제   
 ...      ...                                        ...        ...  ..   
 9837  179105    버스 노약자보호석을 내리는 문이 보이는곳, 내리는문에 가까운 곳에 지정 2017-08-31  교통   
 9838  179106                             지하철 임산부 지정석 관련 2017-08-31  교통   
 9839  179107                    채동선 작곡가의 서울 집을 보존해 주세요. 2017-08-31  문화   
 9840  179108                               한강및 지천에서의 안전 2017-08-31  환경   
 9841  179110       한강 시민공원 주차장에 태양광 패널로 지붕을 설치하면 좋겠습니다. 2017-08-31  환경   
 
                                                      내용  \
 0     운전을 하다보면 횡단보도 주변에 적치물들 때문에 사각지대

In [61]:
## Save pickle
with open("sample2.pickle","wb") as fw:
    pickle.dump([train, test2], fw)
